In [1]:
import os
import pdfplumber
import konlpy
import re
import PIL
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import numpy as np
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
from gensim.models import Word2Vec
import random
from IPython.display import clear_output
import time

C:\Users\1004t\anaconda3\Lib\site-packages\botocore\utils.py:15: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


# 연관 분석

In [2]:
# 페이지 별로 단어 그룹화
json_name = 'pdf_data.json'
with open(json_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

transactions = []

for pages in data.values():
    for nouns in pages.values():
        tmp = []
        for noun in nouns['noun']:
            if noun:
                tmp.extend(noun)
        transactions.append(tmp)
print('Size of transactions : ', len(transactions))
        
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_array, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.6, num_itemsets=len(frequent_itemsets))

Size of transactions :  474


In [3]:
rules_sort = rules.sort_values('lift', ascending=False)

# Word2Vec

In [4]:
# with open('pdf_data.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)
word2vecList = []
for page in data.values():
    for nouns in page.values():
        for noun in nouns['noun']:
            if noun:
                word2vecList.append(noun)
model = Word2Vec(word2vecList, vector_size=100, window=5, min_count=1, workers=4)

In [5]:
model.wv.similarity('데이터', '분석')

0.6416816

# 꼬맨틀

In [6]:
# 단어 목록 생성
nounList = []
for nouns in word2vecList:
    for noun in nouns:
        nounList.append(noun)
nounList = set(nounList)
len(nounList)

973

In [8]:
# 정답 단어 선정
def answer_word():
    filtered_words = [word for word in nounList if len(word) >= 2]

    if filtered_words:  # 필터링 결과가 비어 있지 않은 경우
        random_word = random.choice(filtered_words)
    return random_word

In [9]:
# 유사 단어 순위 매기기
def ranking(answer):
    ranking = []

    for noun in nounList:
        similar = model.wv.similarity(noun, answer)
        ranking.append({noun : similar})

    # 내림차순 정렬
    sorted_ranking = sorted(ranking, key=lambda x: list(x.values())[0], reverse=True)
    sorted_ranking = sorted_ranking[:100]
    return sorted_ranking

In [10]:
# 유사도 계산 및 순위 출력
def calc_similarity(answer, word):
    rank = None
    try:
        similarity = model.wv.similarity(answer, word)
    except:
        print('데이터셋에 없는 단어입니다.')
        return None, None
    for index, dictionary in enumerate(sorted_ranking):
        if dictionary.get(word, None) == similarity:
            rank = index + 1
            break
    return rank, similarity * 100

In [11]:
# 특정 단어에 대해 포함된 PDF 파일 이름을 찾는 함수
def find_pdfs_containing_word(data, word):
    pdfs_with_word = []
    for pdf, pages in data.items():
        for page, content in pages.items():
            # Check if the word exists in the 'noun' list
            for noun_list in content['noun']:
                if word in noun_list:
                    pdfs_with_word.append(pdf)
                    break  # Move to the next PDF once the word is found
    return list(set(pdfs_with_word))  # Remove duplicates

In [12]:
answer = answer_word()
sorted_ranking = ranking(answer)
pdf_hint = find_pdfs_containing_word(data, answer)

# 입력 단어 기록
used_words = set()
history = []
c = 1
latest=''


while True:
    clear_output()  # 이전 출력 내용 삭제
    print("Let's play 경맨틀!!!")
    
    # 결과 히스토리 출력
    tenth = [float(value) for value in sorted_ranking[9].values()]
    fifty = [float(value) for value in sorted_ranking[49].values()]
    print(f'10번째로 유사한 단어의 유사도는 {tenth[0]*100:.2f}이고 50번째로 유사한 단어의 유사도는 {fifty[0]*100:.2f} 입니다.')
    print("\n=== 입력한 단어의 기록 ===")
    for h_c, h_word, h_rank, h_similarity in sorted(history, key=lambda x: x[3], reverse=True):
        rank_text = "None" if h_rank is None else h_rank
        print(f'# {h_c} 단어: {h_word}, 순위: {rank_text}, 유사도: {h_similarity:.3f}')
        
    print(latest)
    print('-'*50)
    
    # 사용자 입력
    word = input('입력 단어 : ').strip()
    if not word:
        print("단어를 입력해주세요!")
        time.sleep(2)
        continue
        
    if word == '포기':
        print(f"정답은 '{answer}'입니다.")
        break
        
    if word == '힌트':
        hint = random.choice(pdf_hint)
        print(f'정답 단어는 {hint} 파일에 있을 수도 있습니다.')
        time.sleep(2)
        continue
        
    if word in used_words:
        print(f'"{word}"는 이미 입력한 단어입니다. 다른 단어를 입력해주세요!')
        # 연관된 단어 검색 및 출력
        related_rules = rules_sort[rules_sort['antecedents'].apply(lambda x: word in x)]
        if not related_rules.empty:
            print(f'입력한 단어 "{word}"와 연관된 단어:')
            for _, row in related_rules[:3].iterrows():
                print(f" - {row['consequents']} (신뢰도: {row['confidence']:.2f})")
        else:
            print(f'입력한 단어 "{word}"와 연관된 단어가 없습니다.')
        time.sleep(2)
        continue

    # 유사도 계산
    rank, similarity = calc_similarity(answer, word)
    if similarity is not None:
        used_words.add(word)
        history.append((c, word, rank, similarity))
        c += 1
        latest = f'입력한 단어 "{word}"의 순위는 {rank}, 유사도는 {similarity:.3f} 입니다.'
        print(latest)
        print('-'*50)
        # 정답 체크
        if word == answer:
            print(f"축하합니다! 정답은 '{answer}' 입니다!")
            break
        
        # 연관된 단어 검색 및 출력
        related_rules = rules_sort[rules_sort['antecedents'].apply(lambda x: word in x)]
        if not related_rules.empty:
            print(f'입력한 단어 "{word}"와 연관된 단어:')
            for _, row in related_rules[:3].iterrows():
                print(f" - {row['consequents']} (신뢰도: {row['confidence']:.2f})")
        else:
            print(f'입력한 단어 "{word}"와 연관된 단어가 없습니다.')

        print('-' * 50)
        time.sleep(2)
    else:
        time.sleep(2)

Let's play 경맨틀!!!
10번째로 유사한 단어의 유사도는 24.12이고 50번째로 유사한 단어의 유사도는 18.21 입니다.

=== 입력한 단어의 기록 ===
# 1 단어: 유사한, 순위: None, 유사도: 4.644
입력한 단어 "유사한"의 순위는 None, 유사도는 4.644 입니다.
--------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [115]:
# 단어 리스트
for i in range(0, len(word2vecList), 10):
    print(word2vecList[i:i+10])

['유사한', '패키지', '연산', '퍼셉트론', '문제', '텍스트', '복사', '공백', '중첩', '명령어']
['생성', '수집', '간주', '분류표', '저축률', '조정해', '결정', '추세', '기존', '아무것']
['시냅스', '목표', '내장', '정상', '분류기', '빈번', '자료', '세포', '시리즈', '성립']
['경우', '산만', '파생변수', '데', '축소', '분석하기', '이웃', '증가', '지점', '도달']
['계층', '경향성', '직사각형', '워드', '빈도', '집단별', '날짜', '검토', '별도', '나열']
['반', '검색하기', '대통령의', '막대', '이하', '나누기', '신경', '라', '마케팅', '정규']
['제조', '번호', '출마', '표현함', '아래', '조합해', '대문자', '개체들', '판단', '자동차들']
['열', '코드들', '부여', '동사', '군집', '만족', '메서드', '이름', '특정', '실제']
['수영', '구조', '가격', '여성', '군집분석', '문', '하나', '정의', '백분율', '정제됨']
['언어', '격', '표현', '구성', '정확', '조합', '전역최적해', '정리하기', '문장', '중심']
['처음', '특성', '둠', '반환', '결과', '추정', '해상', '사후', '랜덤포레스트', '평가']
['방식', '전륜구동', '전', '커널', '몇', '괄호', '사륜구동', '정제', '수평', '공동']
['데이터마이닝', '분류', '확인하기', '특징', '이중분할', '나', '따름', '년', '설치', '상태']
['크기순', '종', '표시하기', '인코딩', '개발', '정렬하세요', '환율', '대부분', '사분위', '정제하기']
['연설문', '오름차', '충분한', '속성', '부족', '하강법', '후륜구동', '병합형', '원리', '꼬꼬마']
['부재', '측면', '네이버책